<a href="https://colab.research.google.com/github/ulubeykhuja/mohir_dev_lessons/blob/main/konveyr_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MLga tayyorgarlik. Pipeline. Sonlar**

Pipeline = konveyr

In [1]:
import pandas as pd
import numpy as np
import sklearn

# Onlayn datasetni ochish
URL = "https://raw.githubusercontent.com/ageron/handson-ml2/refs/heads/master/datasets/housing/housing.csv"
df = pd.read_csv(URL)
df.head()

# trein va test ga bo'lish
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)

# train uchun bashorat qilinadigan qator (median_house_value)ni tashlab yuborish
housing = train_set.drop("median_house_value", axis = 1)
# bashorat qilinadigan qator (median_house_value)ni alohida saqlash
housing_labels = train_set["median_house_value"].copy()

# son bo'magan qatorni tashlash
housing_num = housing.drop("ocean_proximity", axis=1)

In [3]:
# Transformer yasash
from sklearn.base import BaseEstimator, TransformerMixin
# bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [6]:
# Konveyr yasash uchun library
from sklearn.pipeline import Pipeline
# malumotlar bir nechta jarayonlardan otdi
# kerakli lib larni chaqiraiz
from sklearn.impute import SimpleImputer # medianlar uchun class
from sklearn.preprocessing import OneHotEncoder, StandardScaler #

# Sonli ustunlar uchun alohida konveyr yasash
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("attribs_adder", CombinedAttributesAdder()),
    ("std_scaler", StandardScaler())
])

# chaqirish
num_pipeline.fit_transform(housing_num)

array([[ 1.27258656, -1.3728112 ,  0.34849025, ..., -0.17491646,
         0.05137609, -0.2117846 ],
       [ 0.70916212, -0.87669601,  1.61811813, ..., -0.40283542,
        -0.11736222,  0.34218528],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.08821601,
        -0.03227969, -0.66165785],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ..., -0.60675918,
         0.02030568,  0.99951387],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.40217517,
         0.00707608, -0.79086209],
       [-1.41489815,  0.99543676,  1.85617335, ..., -0.85144571,
        -0.08535429,  1.69520292]])

In [ ]:
#housing_onehot = pd.get_dummies(housing['ocean_proximity'])
# housing_onehot

In [9]:
# Matnli konveyr yasash
from sklearn.compose import ColumnTransformer
num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

#Katta konveyr
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs), # num_pipeline ni full konveyrga solish
    # 3qismga bo'lindi, ustunlar ajratildi
    ("cat", OneHotEncoder(), cat_attribs)
])

In [10]:
list(housing_num)

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income']

In [17]:
full_pipeline.fit_transform(housing)

array([[ 1.27258656, -1.3728112 ,  0.34849025, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813, ...,  0.        ,
         0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.        ,
         0.        ,  0.        ],
       [-1.41489815,  0.99543676,  1.85617335, ...,  0.        ,
         1.        ,  0.        ]])

In [16]:
housing_prepared = full_pipeline.fit_transform(housing)

havola : https://colab.research.google.com/github/MohirdevTexnik/MachineLearning_Noutbook/blob/main/05-ml-03-Pipeline.ipynb